# Example 12: Physical Engineering Audit (FAIR Metrics)

This example focuses on the **Facts** layer of the diagnostic suite.
Unlike Q-Scores (which are calibrated interpretations), FAIR metrics provide raw, physical measurements of the population's health.

We simulate a **Premature Convergence** scenario where the algorithm has reached the Pareto surface but hasn't yet spread out uniformly, leaving large gaps and irregular clusters.

In [ ]:
from MoeaBench import mb
import numpy as np
import matplotlib.pyplot as plt

print("MoeaBench Imported.")

### 1. Premature Convergence Setup
We use DTLZ2 (3 objectives) and NSGA-II, but run it for only **30 generations**. At this stage, the population is usually close to the target surface but highly clustered.

In [ ]:
# 1. Setup
mop = mb.mops.DTLZ2(M=3)
gt = mop.pf(n_points=500)

exp = mb.experiment()
exp.mop = mop
exp.moea = mb.moeas.NSGA2(population=100, generations=30)

print("Running optimization (Scenario: Premature Convergence)...")
exp.run()

### 2. Physical Evidence (Individual Metrics)
We look at raw physical values like mean distance to the front (Closeness) and the largest hole size (Gap).

In [ ]:
# A. Closeness Distribution
u_dist = mb.diagnostics.fair_closeness(exp, ground_truth=gt)
print(f"Mean Distance: {np.mean(u_dist):.4f}")

# B. Coverage & Gaps
f_cov = mb.diagnostics.fair_coverage(exp, ground_truth=gt)
f_gap = mb.diagnostics.fair_gap(exp, ground_truth=gt)
print(f"Coverage (Physical): {f_cov:.4f}")
print(f"Largest Gap: {f_gap:.4f}")

### 3. Full Physical Audit
The consolidated audit provides a structured report of all engineering facts.

In [ ]:
mb.diagnostics.fair_audit(exp, ground_truth=gt).report_show()

### 4. Topology Visualization
The visual confirmation shows the 'Lattice' quality vs Reality.

In [ ]:
mb.view.topo_shape(exp, gt, 
                   title="Physical Pathology: Premature Convergence",
                   labels=["Early Population", "Optimal Front (GT)"])